In [2]:
import numpy as np

N = 8
lattice_basis =  np.eye(N).astype(int)

print(lattice_basis)

[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]]


In [3]:
list_bn = [
827733596782305800412928022203,
949647717115293590693294029007,
513755983108074989775373368979,
677679034424075998463761252321,
453168678084622963159703254177,
832782910923539364543910431379,
252562957778396321268184912171,
651496223062243390455276796757,
432919332491435708844136579631,
223770585052094118295318801799,
532148703589048376478494845573,
243856773163704399644951505359,
164819263334667219851593378483,
401469735249463137163038448637,
426751380869555974285167310687,
560854616065731349966291565407,
683800733848288692548599026941,
720403676000154697037384804609,
107301258962892384849699294203,
520071830033615222504245399291,
994869210101506594790719948613,
245219053806416037970010902657,
686041058007528223688600531911,
627290270131015070770970266369,
986406199575879090372513368911,
161242727312477951737327198843,
774858181281373313211111520339,
684559254088515873739510084459,
257694642268698126951808240063,
571233875063041256502141730223,
644801812715761815209851630957,
959697197099941066839436226867,
624429478423882721953553862137,
174367152486141929170585530673,
599558656796597046233579141381,
433361790406333033396425865133,
327914293338207349929854266927,
469829830404491409659825996297,
440912965020782384634858339911,
117716796822761004609505655351,
202743088259468212450682455949,
825678387625860526264958140667,
186563286821803413392902752181,
344598372208490315697621082241,
250491666241092516038026895957,
455905910983924930564630078541,
764508361786508023846708165651,
368402894141896908753430052087,
632529273538568546719862413771,
238262282320450157628785827507,
527663446629057224413918255997,
733182256888608076021693398743,
294525687336161906252161965089,
168418874893897148434218110009,
967500129217854143886182418299,
918282852294568912813037878951,
351474820193810768760797418683,
472054784060326913010893152441,
213047323530044816411212536661,
840631896797768973136803074339,
994618666924895047842532997701,
696690223952403242478248080393,
302823001073534810462474580767,
415854418638180287264172792169]

# slice the list to N portions
sliced_list_bn = [list_bn[i::N] for i in range(N)]

print(len(sliced_list_bn))

8


In [4]:
list_basis = []
for i in range(N):
    for j in range(N):
        list_basis.append(sliced_list_bn[i][j] * lattice_basis[i][j])


modulo = 2147483647 # Mersenne prime 2^31 - 1
print(modulo)


for i in range(len(list_basis)):
    list_basis[i] = list_basis[i] % modulo

sliced_list_basis = [list_basis[i::N] for i in range(N)]
print(len(sliced_list_basis))


2147483647
8


In [5]:
def matrix_eye(N):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        result[i][i] = int(1)
    return result

In [6]:
secret = [[[np.random.randint(-10, 10) for _ in range(N)] for _ in range(N)] for _ in range(N)]
shared_secret = [[] for _ in range(N)]
public = [(matrix_eye(N), matrix_eye(N)) for _ in range(N)]
public_cache = [([], []) for _ in range(N)]
public_rep = [([], []) for _ in range(N)]
public_cache_rep = [([], []) for _ in range(N)]

print(len(secret))
print(public_rep)
print(public_cache_rep)

8
[([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], [])]
[([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], []), ([], [])]


In [7]:
def matrix_mul(matrix_a, matrix_b, modulo):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                result[i][j] += matrix_a[i][k] * matrix_b[k][j]
                result[i][j] %= modulo
    return result

In [8]:
def matrix_add(matrix_a, matrix_b, modulo):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            result[i][j] = matrix_a[i][j] + matrix_b[i][j]
            result[i][j] %= modulo
    return result

In [9]:
def matrix_mul_and_add(matrix_a, matrix_b, matrix_c, modulo):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                result[i][j] += matrix_a[i][k] * matrix_b[k][j]
                result[i][j] %= modulo
            result[i][j] += matrix_c[i][j]
            result[i][j] %= modulo
    return result

In [10]:
def matrix_div(matrix, divisor=100):
    result = [[int(0) for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            result[i][j] = matrix[i][j] // divisor
    return result

In [11]:
def print_matrix(matrix):
    for i in range(N):
        for j in range(N):
            print(f"{int(matrix[i][j]):15}", end=' ')
        print()
    print()

In [12]:
a = matrix_mul(secret[0], sliced_list_basis, modulo)
a = matrix_mul(a, secret[1], modulo)
a = matrix_mul(a, sliced_list_basis, modulo)
a = matrix_mul(a, secret[2], modulo)

e = matrix_mul(sliced_list_basis, secret[2], modulo)
e = matrix_mul(secret[1], e, modulo)
e = matrix_mul(sliced_list_basis, e, modulo)
e = matrix_mul(secret[0], e, modulo)

for i in range(N):
    for j in range(N):
        print(f"{int(a[i][j]):15}", end=" ")
    print()

print()

for i in range(N):
    for j in range(N):
        print(f"{int(e[i][j]):15}", end=" ")
    print()

      635622720      1818713396       499758460      2007078892       274372908       845061405      1227367490       145364168 
      393015411      1219316105      1707290408       543633849      2073376966      1016654639       163979379      1800950738 
       85836783       616514951       816603876       145929562      1933290771       835464047      2036923906      1393500173 
     1887011667      1761164371      1505885351       853113260       752660241      2131046552      1723260999      1344491362 
     1598057234      1887567481       628320681      1339883790       722892666       906527547       906130669       465573942 
     2077465106      1810833866      1178278766      2117134868       992544733       244183370      1572581843      1137606395 
     1437897052       218748198       242507657       773704089       717232719      1532181602      2086683088      1583566181 
     1037081224      1827282634      1997905352      1940778859      1252384076       970737990  

In [13]:
def public_compose(secret_matrix, public_matrix_pair, id: int, left_or_right: int, modulo):
    if left_or_right == 0:
        a = matrix_mul(secret_matrix, sliced_list_basis, modulo)
        a = matrix_mul(public_matrix_pair[0], a, modulo)
        return (a, public_matrix_pair[1])
    else:
        a = matrix_mul(sliced_list_basis, secret_matrix, modulo)
        a = matrix_mul(a, public_matrix_pair[1], modulo)
        return (public_matrix_pair[0], a)

In [14]:
def public_compose_init(secret_matrix, id: int, left_or_right: int, modulo):
    if left_or_right == 0:
        a = matrix_mul(secret_matrix, sliced_list_basis, modulo)
        return (a, matrix_eye(N))
    else:
        a = matrix_mul(sliced_list_basis, secret_matrix, modulo)
        return (matrix_eye(N), a)

In [15]:
def share_secret_compose(secret_matrix, public_matrix_pair, n: int, modulo):
    a = matrix_mul(public_matrix_pair[0], secret_matrix, modulo)
    a = matrix_mul(a, public_matrix_pair[1], modulo)
    return a

In [16]:
def multi_DH(n, round=0, verbose=False):
    '''
    This function computes the multi-party Diffie-Hellman key exchange protocol.
    n: number of parties (power of 2)
    round: the current round in current subset n
    prefix: the prefix of the current round
    total_rounds: the total number of rounds 
    '''
    def pass_round(n, id0, id1, from_id0, from_id1):
        public[id0] = public_compose(secret[id0], public[from_id0], id0, 0, modulo)
        public[id1] = public_compose(secret[id1], public[from_id1], id1, 1, modulo)
        public_rep[id0] = (public_rep[from_id0][0][:], public_rep[from_id0][1][:])
        public_rep[id0][0].append(id0)
        public_rep[id1] = (public_rep[from_id1][0][:], public_rep[from_id1][1][:])
        public_rep[id1][1].insert(0, id1)
        if verbose:
            print(f"Public {id0}: {public_rep[id0]}")
            print(f"Public {id1}: {public_rep[id1]}")
        return
    
    def pass_round_from_cache(n, id0, id1, from_id):
        public[id0] = public_compose(secret[id0], public_cache[from_id], id0, 0, modulo)
        public[id1] = public_compose(secret[id1], public_cache[from_id], id1, 1, modulo)
        public_rep[id0] = (public_cache_rep[from_id][0][:], public_cache_rep[from_id][1][:])
        public_rep[id0][0].append(id0)
        public_rep[id1] = (public_cache_rep[from_id][0][:], public_cache_rep[from_id][1][:])
        public_rep[id1][1].insert(0, id1)
        if verbose:
            print(f"Public {id0}: {public_rep[id0]}")
            print(f"Public {id1}: {public_rep[id1]}")
        return
    
    def cache_public(n, id0, id1):
        public_cache[id0] = public[id0]
        public_cache[id1] = public[id1]
        public_cache_rep[id0] = public_rep[id0]
        public_cache_rep[id1] = public_rep[id1]
        return
    
    if verbose:
        print(f"Round {round}:")
    match round:
        case 0:
            public[0] = public_compose_init(secret[0], 0, 0, modulo)
            public[7] = public_compose_init(secret[7], 7, 1, modulo)
            public_rep[0] = ([0], [])
            public_rep[7] = ([], [7])
        case 1:
            pass_round(n, 1, 6, 0, 7)
        case 2:
            pass_round(n, 2, 5, 1, 6)
        case 3:
            pass_round(n, 3, 4, 2, 5)
            cache_public(n, 3, 4)
        case 4:
            pass_round_from_cache(n, 0, 3, 4)
            pass_round_from_cache(n, 4, 7, 3)
        case 5:
            pass_round(n, 1, 2, 0, 3)
            pass_round(n, 5, 6, 4, 7)
            cache_public(n, 1, 2)
            cache_public(n, 5, 6)
        case 6:
            pass_round_from_cache(n, 0, 1, 2)
            pass_round_from_cache(n, 2, 3, 1)
            pass_round_from_cache(n, 4, 5, 6)
            pass_round_from_cache(n, 6, 7, 5)
        case 7:
            shared_secret[0] = share_secret_compose(secret[0], public[1], n, modulo)
            shared_secret[1] = share_secret_compose(secret[1], public[0], n, modulo)
            shared_secret[2] = share_secret_compose(secret[2], public[3], n, modulo)
            shared_secret[3] = share_secret_compose(secret[3], public[2], n, modulo)
            shared_secret[4] = share_secret_compose(secret[4], public[5], n, modulo)
            shared_secret[5] = share_secret_compose(secret[5], public[4], n, modulo)
            shared_secret[6] = share_secret_compose(secret[6], public[7], n, modulo)
            shared_secret[7] = share_secret_compose(secret[7], public[6], n, modulo)

In [20]:
for i in range(N):
    print(f"Secret {i}:")
    print("[", secret[i][0][0], ",", secret[i][7][7], "]")

# Run the multi-party Diffie-Hellman key exchange protocol
for i in range(N):
    multi_DH(N, round=i, verbose=True)
    for j in range(N):
        print(f"Public {i} {j}:")
        print("(", public[j][0][0][0], ",", public[j][1][0][0], ")")

for i in range(N):
    print(f"Shared Secret {i}:")
    print_matrix(shared_secret[i])

Secret 0:
-9
Secret 1:
4
Secret 2:
-5
Secret 3:
8
Secret 4:
-9
Secret 5:
7
Secret 6:
-10
Secret 7:
-8
Round 0:
Public 0 0:
( 1521774584 , 1 )
Public 0 1:
( 1 , 381069642 )
Public 0 2:
( 667256716 , 2023310931 )
Public 0 3:
( 1007969992 , 355282260 )
Public 0 4:
( 1696133860 , 420287826 )
Public 0 5:
( 1649382548 , 1579522981 )
Public 0 6:
( 217680102 , 1 )
Public 0 7:
( 1 , 1114079225 )
Round 1:
Public 1: ([0, 1], [])
Public 6: ([], [6, 7])
Public 1 0:
( 1521774584 , 1 )
Public 1 1:
( 1007969992 , 1 )
Public 1 2:
( 667256716 , 2023310931 )
Public 1 3:
( 1007969992 , 355282260 )
Public 1 4:
( 1696133860 , 420287826 )
Public 1 5:
( 1649382548 , 1579522981 )
Public 1 6:
( 1 , 420287826 )
Public 1 7:
( 1 , 1114079225 )
Round 2:
Public 2: ([0, 1, 2], [])
Public 5: ([], [5, 6, 7])
Public 2 0:
( 1521774584 , 1 )
Public 2 1:
( 1007969992 , 1 )
Public 2 2:
( 667256716 , 1 )
Public 2 3:
( 1007969992 , 355282260 )
Public 2 4:
( 1696133860 , 420287826 )
Public 2 5:
( 1 , 1579522981 )
Public 2 6:
(